In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import json

In [3]:
with open('../utils.json', 'r') as f:    
    json_data = json.load(f)
    file_path = json_data['data']['path']

with h5py.File(file_path, 'r') as h5f:
    traces = h5f['traces'][()]
    plaintexts = h5f['plaintext'][()]
    ciphertexts = h5f['ciphertext'][()]
    keys = h5f['keys'][()]

### Differential Power Analysis (DPA)

1. 选择一个泄露模型.
2. 找到信噪比最高的点.
3. 设置一个Threshold, 遍历可能的密钥, 计算中间值通过Threshold把能量迹分为两组,均值做差.
4. 选择差分最大的密钥作为K.

\begin{align}

\triangle D[j] & = \frac{\sum_{i = 1}^{m}D(C_i,b,K_s)\textbf{T}_i(j)}{\sum_{i = 1}^{m}D(C_i,b,K_s)} - 
\frac{\sum_{i = 1}^{m}(1-D(C_i,b,K_s))\textbf{T}_i(j)}{\sum_{i = 1}^{m}(1-D(C_i,b,K_s))}   \\
&\approx 2(\frac{\sum^{m}_{i=1}D(C_i,b,K_s)\textbf{T}_i[j]}{\sum ^{m}_{i=1}D(C_i, b, K_s)} - \frac{\sum ^{m}_{i=1}\textbf{T}_i[j]}{m} )
\end{align}

In [ ]:
import mlscat as cat
from tqdm import tqdm
def dpa(traces, plaintexts, threshold, target_byte, target_point, leakage_function):
    """
    DPA Attack

    This function is just for AES-128, if attack AES-256 or others, plz change it

    Parameters
    ----------
    traces : array
    plaintexts : array
    threshold : int
    target_byte : int
    target_point : int
    leakage_function : str

    Returns
    -------
    after calculation get the maximum value return candidate_key
    
    Demo:
    ----------
    >>> python
    ### trace.shape = (2000, 15000), plaintext.shape = (2000, 16)
        dpa(traces, plaintexts, threshold=4, target_byte=0, target_point=810, leakage_function='hw')
    """
    candidate_key = []
    maximum = 0
    mean_diffs = np.zeros(256)
    for i in tqdm(range(256)):
        traces_group1 = []
        traces_group2 = []
        for num in range(len(traces)):
            mid_val = cat.leakage.hw(cat.utils.AES_Sbox[plaintexts[num][target_byte] ^ i]) if leakage_function.lower() == 'hw' else cat.utils.AES_Sbox[plaintexts[num][target_byte] ^ i]
            if mid_val < threshold:
                traces_group1.append(traces[num][target_point])
            else:
                traces_group2.append(traces[num][target_point])
        mean_diffs[i] = abs(np.mean(traces_group1) - np.mean(traces_group2))
        if mean_diffs[i] > maximum:
            maximum = mean_diffs[i]
            candidate_key = i
    return candidate_key, mean_diffs

In [16]:
targets = []
for num in range(len(traces)):
    targets.append(cat.leakage.hw(cat.utils.AES_Sbox[plaintexts[num][0] ^ keys[num][0]]))
pcc = cat.attacks.pcc(targets, np.array(traces))

In [19]:
dpa(traces, plaintexts, threshold=4, target_byte=0, target_point=810, leakage_function='hw')

100%|██████████| 256/256 [00:01<00:00, 241.25it/s]


(188,
 array([1.02289798e+02, 2.37567535e+02, 1.08856050e+02, 1.37646223e+02,
        3.13101820e+02, 1.41820850e+02, 2.07191204e+02, 6.90420875e+01,
        7.91048335e+01, 5.68372219e+01, 5.51536521e+01, 3.02437717e+01,
        2.18142646e+02, 6.16161719e+01, 5.37984375e+01, 1.18357752e+02,
        1.39378297e+02, 1.04486798e+02, 3.36633582e+02, 2.66961226e+01,
        2.33290452e+01, 3.22982096e+02, 3.27087164e+02, 1.06914401e+02,
        8.16281879e+01, 3.60903758e+01, 6.77693487e+01, 3.97918422e+02,
        1.07274249e+02, 1.40564723e+02, 2.30707882e+02, 3.45196956e+00,
        3.91537444e+02, 4.13893484e+01, 2.36450881e+01, 1.74854391e+02,
        1.60535330e+02, 1.49465713e+02, 2.71419455e+02, 1.12445214e+02,
        2.07418731e+02, 9.65027491e+01, 8.48018496e+01, 5.64825043e+01,
        1.71480221e+02, 8.95829466e+01, 1.02815150e+02, 3.70564222e+01,
        1.33881081e+02, 1.47330708e+02, 6.29193291e+01, 6.04335275e+01,
        4.51779869e+01, 1.67733311e+02, 3.34679208e+01, 1.

In [20]:
keys

array([[188,  52, 154, ..., 242, 101, 223],
       [188,  52, 154, ..., 242, 101, 223],
       [188,  52, 154, ..., 242, 101, 223],
       ...,
       [188,  52, 154, ..., 242, 101, 223],
       [188,  52, 154, ..., 242, 101, 223],
       [188,  52, 154, ..., 242, 101, 223]], dtype=uint8)